In [2]:
import sys
from PySide6 import QtWidgets
from PySide6 import QtCore
from PySide6 import QtGui
from PySide6.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QPushButton
from PySide6.QtSerialPort import QSerialPort, QSerialPortInfo
from PySide6.QtCore import QIODevice, QByteArray
from PySide6.QtCore import QTime, QTimer, Slot
import pyqtgraph as pg
import numpy as np
from time import *
import warnings

import ctypes
myappid = 'nil.npm.pyqt.1' # arbitrary string
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)

from PySide6.QtGui import *
from PySide6.QtWidgets import *
from PySide6.QtCore import *

from importlib import reload

import ui_main
reload(ui_main)
from ui_main import *

import ui_dialog
reload(ui_dialog)
from ui_dialog import *

from serial import *
import serial.tools.list_ports

class Dialog(QDialog, Ui_Dialog): # save position dialog box
    def __init__(self, points, parent=None):
        QDialog.__init__(self, parent)
        self.points = points
        self.setupUi(self)
        self.setWindowTitle("Save Current Position")
        self.setWindowIcon(QtGui.QIcon('save_icon.png'))
        self.buttonBox.accepted.connect(self.on_save)
        self.comboBox.activated.connect(self.on_activate)
        self.comboBox.clear()
        self.comboBox.addItems(self.points)
        self.comboBox.setPlaceholderText('Select Save Point')
        self.selection=self.comboBox.itemText(0)
        self.index=0
        self.save=False
        comboBox=[]
        for i in range(0,self.comboBox.count()):
            comboBox.append(self.comboBox.itemText(i))
        print(str(comboBox), end='\r') 

    def on_activate(self):
        print(self.comboBox.currentText()+'                    ', end='\r')
        self.selection=self.comboBox.currentText()
        self.index=self.comboBox.currentIndex()
    
    def on_save(self):
        self.comboBox.setItemText(self.index, self.name_input.text())
        self.save=True
        
class MainWindow(QtWidgets.QMainWindow, Ui_MainWindow): # main window
    def __init__(self):
        super(MainWindow, self).__init__()
        self.setupUi(self)

        self.setWindowIcon(QtGui.QIcon('nanopen_icon.png'))
        self.setWindowTitle("Nanopen Manipulator")

        self.save_point.clicked.connect(self.on_save_point)
        self.save_point.setFocusPolicy(Qt.NoFocus)
        self.set_zero.clicked.connect(self.on_set_zero)
        self.set_zero.setFocusPolicy(Qt.NoFocus)
        self.step_mode.clicked.connect(self.on_step_mode)
        self.step_mode.setFocusPolicy(Qt.NoFocus)
        self.origin.clicked.connect(self.on_origin)
        self.origin.setFocusPolicy(Qt.NoFocus)
        self.free_mode.clicked.connect(self.on_free_mode)
        self.free_mode.setFocusPolicy(Qt.NoFocus)
        self.stop.clicked.connect(self.on_stop)
        self.stop.setFocusPolicy(Qt.NoFocus)

        self.sp1.clicked.connect(self.on_sp)
        self.sp1.setFocusPolicy(Qt.NoFocus)
        self.sp2.clicked.connect(self.on_sp)
        self.sp2.setFocusPolicy(Qt.NoFocus)
        self.sp3.clicked.connect(self.on_sp)
        self.sp3.setFocusPolicy(Qt.NoFocus)
        self.sp4.clicked.connect(self.on_sp)
        self.sp4.setFocusPolicy(Qt.NoFocus)
        self.sp5.clicked.connect(self.on_sp)
        self.sp5.setFocusPolicy(Qt.NoFocus)

        self.sp1_xyz=[]
        self.sp2_xyz=[]
        self.sp3_xyz=[]
        self.sp4_xyz=[]
        self.sp5_xyz=[]
        self.sp_vars=[self.sp1_xyz,self.sp2_xyz,self.sp3_xyz,self.sp4_xyz,self.sp5_xyz]

        self.points=[self.sp1.text(),self.sp2.text(),self.sp3.text(),self.sp4.text(),self.sp5.text()]
        self.sp_buttons=[self.sp1,self.sp2,self.sp3,self.sp4,self.sp5]
        
        self.length_input.setValue(30)
        self.width_input.setValue(30)
        self.height_input.setValue(10)
        self.length_input.textChanged.connect(self.on_lwh_input)
        self.width_input.textChanged.connect(self.on_lwh_input)
        self.height_input.textChanged.connect(self.on_lwh_input)

        self.step_size.setValue(100)
        self.step_size.textChanged.connect(self.on_step_size)

        self.speed.setValue(100)
        self.speed.valueChanged.connect(self.on_speed)

        self.main_plot.setMouseEnabled(x=True, y=True)  # Disable mouse panning & zooming
        self.main_plot.setFocusPolicy(QtCore.Qt.NoFocus.StrongFocus)
        self.main_plot.hideButtons()  # Disable corner auto-scale button
        self.main_plot.showGrid(x=True, y=True)
        self.main_plot.setLabel("left", "Y (um)")
        self.main_plot.setLabel("bottom", "X (um)")
        self.x=0
        self.y=0
        self.z=0
        self.step_x = 0
        self.step_y = 0
        self.step_z = 0

        self.z_axis.setFocusPolicy(QtCore.Qt.NoFocus.StrongFocus)
        self.z_axis.hideButtons()  # Disable corner auto-scale button
        self.z_axis.hideAxis('bottom')
        self.z_axis.showGrid(x=False, y=True)
        self.z_axis.setLabel("left", "Z (um)")

        self.is_key_W_pressed = False
        self.is_key_A_pressed = False
        self.is_key_S_pressed = False
        self.is_key_D_pressed = False

        self.is_key_SHIFT_pressed=False
        self.is_key_SPACE_pressed=False
        
        self.z_axis.setMouseEnabled(x=False, y=True)  # Disable mouse panning & zooming
        self.z_axis.hideButtons()  # Disable corner auto-scale button

        self.l = 30000 # arbitrary limits
        self.w = 30000
        self.h = 10000

        self.main_plot.setXRange(0, self.l)
        self.main_plot.setYRange(0, self.w)
        self.main_plot.setLimits(xMin=0, yMin=0, xMax=self.l, yMax=self.w)
        self.z_axis.setLimits(yMin=0, yMax=self.h)
        self.z_axis.setXRange(0, 1)
        self.z_axis.setYRange(0, self.h)

        self.step=100
        self.is_step_mode=False

        self.timer = QTimer()
        self.serial_timer = QTimer()
        self.zero=[0,0,0]
        self.i = 0

        self.is_free_mode=False
        self.buffer= b''

        # Set up QSerialPort
        self.serial = QSerialPort(self)
        self.serial.readyRead.connect(self.read_serial_data)
        self.com_port=''
        
        # ports
        self.ports = sorted(serial.tools.list_ports.comports())
        self.serial_port=""
        for i in range(0,len(self.ports)):
            com_port=QAction(self.ports[i][0], self, checkable=True)
            com_port.triggered.connect(self.on_serial)
            self.menuSerial.addAction(com_port)
        for port in self.ports:
            self.serial.setPortName(port[0])
            self.serial.close()

        self.lastData = [0, 0, 0]
        self.serialTimerDelay = 33
    
    def on_serial(self): # select port
        for port in self.menuSerial.actions():
            try:
                if port.isChecked():
                    # Configure and open the serial port
                    self.com_port = port.text()
                    self.serial.setPortName(self.com_port)  # Change this to your port
                    self.serial.setBaudRate(QSerialPort.Baud9600)
                    self.serial.setDataBits(QSerialPort.Data8)
                    self.serial.setParity(QSerialPort.NoParity)
                    self.serial.setStopBits(QSerialPort.OneStop)
                    self.serial.setFlowControl(QSerialPort.NoFlowControl)
                if not self.serial.open(QIODevice.ReadWrite) or not port.isChecked():
                    self.serial_timer.stop()
                    self.serial_timer.timeout.disconnect(self.start_Serial_Move)
                    port.setChecked(False)
                    self.serial.close()
                    print(f"Closed port: {self.com_port}".ljust(200))  # Confirm port is closed
                    self.connection.setText(QCoreApplication.translate("MainWindow", u"<html><head/><body><p align=\"center\"><span style=\" font-size:8pt;\">Status: Disconnected</span></p></body></html>", None))
                else:
                    if not self.serial_timer.isActive():
                        self.serial_timer.timeout.connect(self.start_Serial_Move)
                        self.serial_timer.start(self.serialTimerDelay)
                        print(f"Opened port: {self.com_port}".ljust(200))  # Confirm port is open
                        self.connection.setText(QCoreApplication.translate("MainWindow", u"<html><head/><body><p align=\"center\"><span style=\" font-size:8pt;\">Status: Connected</span></p></body></html>", None))
                    break
            except Exception as e:
                port.setChecked(False)
                self.connection.setText(QCoreApplication.translate("MainWindow", f'<html><head/><body><p align=\"center\"><span style=\" font-size:8pt;\">Status: {str(e)}</span></p></body></html>', None))
                print(str(e) + ' '*200, end='\r')

    def read_serial_data(self): # print serial output from arduino
        try:
            # Read all available data from the serial port
            incoming_data = self.serial.readAll().data()
            if not incoming_data:
                return
                # Decode data from bytes to string
            self.buffer += incoming_data
            while b'\n' in self.buffer:
                line, self.buffer = self.buffer.split(b'\n', 1)
                decoded_data = line.decode('utf_8')
                print(f'ARDUINO: {decoded_data.strip()}'.ljust(200), end='\r')  # Print the raw data for debugging
                self.print.setText(QCoreApplication.translate("MainWindow", f'<html><head/><body><p align=\"center\"><span style=\" font-size:8pt;\">{decoded_data.strip()}</span></p></body>'))
                if "C" in decoded_data:
                    coords = decoded_data[1:].split(';',3)
                    if len(coords)==3:
                        try:
                            self.x_actual=int(coords[0])
                            self.y_actual=int(coords[1])
                            self.z_actual=int(coords[2])
                        except:
                            pass
                        # clear plots
                        self.main_plot.clear()
                        self.z_axis.clear()
                        try:
                            self.main_plot.plot([self.x], [self.y], pen=None, symbol='o')  # setting pen=None disables line drawing
                            self.z_axis.plot([0], [self.z], pen=None, symbol='o')
                            self.z_axis.plot([0], [self.z_actual], pen=None, symbol='x') 
                            self.main_plot.plot([self.x_actual], [self.y_actual], pen=None, symbol='x')  # setting pen=None disables line drawing
                        except ValueError as ve:
                            print(f"Error converting coordinates: {ve}")
                if "L" in decoded_data:
                    limit = decoded_data[1:].split(';',3)
                    self.l=int(limit[0].split('.',1)[0])
                    self.w=int(limit[1].split('.',1)[0])
                    self.h=int(limit[2].split('.',1)[0])
                    self.x=0
                    self.y=0
                    self.z=0
                    self.main_plot.setLimits(xMax=self.l, yMax=self.w)
                    self.main_plot.setXRange(0, self.l) # 160 steps/mm
                    self.main_plot.setYRange(0, self.w)
                    self.z_axis.setLimits(yMax=self.h)
                    self.z_axis.setYRange(0, self.h)
                    self.length_input.setValue(self.l/1000)
                    self.width_input.setValue(self.w/1000)
                    self.height_input.setValue(self.h/1000)
        except Exception as e:
            print(f"Error reading serial data: {e}")

    def on_free_mode(self, checked): # ignores reference frame
        self.is_free_mode=checked

    def on_stop(self):
        try:
            if self.serial.isOpen():
                self.serial.write('s\n'.encode('utf_8'))
        except Exception as e: print(str(e).ljust(200), end='\r')        

    def on_sp(self): # save points
        button = self.sender().objectName()
        try:
            if button == 'sp1':
                self.x=self.sp1_xyz[0]
                self.y=self.sp1_xyz[1]
                self.z=self.sp1_xyz[2]
            elif button == 'sp2':
                self.x=self.sp2_xyz[0]
                self.y=self.sp2_xyz[1]
                self.z=self.sp2_xyz[2]     
            elif button == 'sp3':
                self.x=self.sp3_xyz[0]
                self.y=self.sp3_xyz[1]
                self.z=self.sp3_xyz[2]     
            elif button == 'sp4':
                self.x=self.sp4_xyz[0]
                self.y=self.sp4_xyz[1]
                self.z=self.sp4_xyz[2]     
            elif button == 'sp5':
                self.x=self.sp5_xyz[0]
                self.y=self.sp5_xyz[1]
                self.z=self.sp5_xyz[2]            
        except:
            pass
        finally:
            self.main_plot.clear()
            self.z_axis.clear()
            self.x_display.display(self.x)
            self.y_display.display(self.y)
            self.z_display.display(self.z)
            self.z_axis.plot([0], [self.z], pen=None, symbol='o')
            self.main_plot.plot([self.x], [self.y], pen=None, symbol='o')  # setting pen=None disables line drawing
            try:
                if self.serial.isOpen():
                    self.serial.write(f'{self.x};{self.y};{self.z}\n'.encode('utf_8'))
            except Exception as e: print(str(e).ljust(200), end='\r')            
            
    def on_origin(self): # origin
        self.main_plot.clear()
        self.z_axis.clear()
        self.x=0
        self.y=0
        self.z=0
        self.x_display.display(self.x)
        self.y_display.display(self.y)
        self.z_display.display(self.z)
        self.z_axis.plot([0], [self.z], pen=None, symbol='o')
        self.main_plot.plot([self.x], [self.y], pen=None, symbol='o')  # setting pen=None disables line drawing
    
    def start_move(self):
        # Check if the timer is already running
        if not self.timer.isActive():
            # Connect the timeout signal to the function only if it's not already connected
            # This prevents multiple connections if start_move is called multiple times
            self.timer.timeout.connect(self.WASDMove)
            # Start the timer with an interval of 1000 milliseconds (1 second)
            self.timer.start(33)
        
    def stop_move(self):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Failed to disconnect")
            # .. your divide-by-zero code ..
            # Stop the timer
            # Disconnect the signal to prevent a potential memory leak
            try:
                self.timer.stop()
                self.timer.timeout.disconnect(self.WASDMove)              
            except:
                # No connection to disconnect, pass
                pass

    def start_Serial_Move(self):
        try:
            if self.serial.isOpen():
                if self.x_actual != self.x or self.y_actual != self.y or self.z_actual != self.z:
                    if self.x != self.lastData[0] or self.y != self.lastData[1] or self.z != self.lastData[2]:
                        self.serial.write(f'P{self.x};{self.y};{self.z}\n'.encode('utf_8'))
                        self.lastData = [self.x, self.y, self.z]
                        #self.serial_timer.setInterval((self.speed.value()/100)^2*self.serialTimerDelay)
                    if self.i == 0:
                        print('CATCH UP BITCH..'.ljust(200), end = '\r')
                        self.i+=1
                    elif self.i == 1:
                        print('CATCH UP BITCH...'.ljust(200), end = '\r')
                        self.i+=1
                    elif self.i == 2:
                        self.i=0
        except Exception as e: print(str(e).ljust(200), end='\r')

    def sendSerialPositions(self):
        try:
            if self.serial.isOpen():
                self.serial.write(f'P{self.x};{self.y};{self.z}\n'.encode('utf_8'))
        except Exception as e: print(str(e).ljust(200), end='\r')
    
    def on_save_point(self): # save point
        self.points=[self.sp1.text(),self.sp2.text(),self.sp3.text(),self.sp4.text(),self.sp5.text()]
        self.stop_move()
        self.Dialog = Dialog(self.points)
        self.Dialog.exec()
        if self.Dialog.save:
            self.sp_buttons[self.Dialog.index].setText(QCoreApplication.translate("MainWindow", self.Dialog.name_input.text(), None))
            self.sp_vars[self.Dialog.index]=[self.x,self.y,self.z]
            print(self.Dialog.index)
            if self.Dialog.index == 0:
                self.sp1_xyz=[self.x,self.y,self.z]
            elif self.Dialog.index == 1:
                self.sp2_xyz=[self.x,self.y,self.z]
            elif self.Dialog.index == 2:
                self.sp3_xyz=[self.x,self.y,self.z]
            elif self.Dialog.index == 3:
                self.sp4_xyz=[self.x,self.y,self.z]
            elif self.Dialog.index == 4:
                self.sp5_xyz=[self.x,self.y,self.z]
            print(self.Dialog.selection+' changed to: '+self.Dialog.name_input.text().ljust(200),end='\r')
        else:
            print('canceled'.ljust(200),end='\r')
        self.is_key_W_pressed=False
        self.is_key_A_pressed=False
        self.is_key_S_pressed=False
        self.is_key_D_pressed=False
        self.is_key_SHIFT_pressed=False
        self.is_key_SPACE_pressed=False

    def on_set_zero(self):
        self.serial.write("H\n".encode('utf_8')) 

    def on_step_mode(self, checked):
        self.is_step_mode=checked
        print('step mode '+str(checked).ljust(200), end='\r') 

    def on_lwh_input(self):
        self.l = int(self.length_input.text())*1000
        self.w = int(self.width_input.text())*1000
        self.h = int(self.height_input.text())*1000
        self.main_plot.setLimits(xMax=self.l, yMax=self.w)
        self.z_axis.setLimits(yMax=self.h)
        self.main_plot.setXRange(0, self.l)
        self.main_plot.setYRange(0, self.w)
        self.z_axis.setXRange(0, 1)
        self.z_axis.setYRange(0, self.h)
        print(str([self.l,self.w,self.h]).ljust(200), end='\r')

    def on_speed(self):
        print(str(self.speed.value()/100).ljust(200), end='\r')

    def on_step_size(self):
        self.step=int(self.step_size.text())
        print(str(self.step).ljust(200), end='\r')

    def keyPressEvent(self, event): 
        if self.is_step_mode: # step mode
            self.is_key_W_pressed = False # reset bools
            self.is_key_A_pressed = False
            self.is_key_S_pressed = False
            self.is_key_D_pressed = False
            self.is_key_SHIFT_pressed=False
            self.is_key_SPACE_pressed=False
            self.stop_move()
            self.start_Serial_Move()
            if event.key() == QtCore.Qt.Key_W and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.y+=self.step
                elif self.y>=self.w or self.y+self.step>=self.w: # boundary conditions
                    self.y=self.w # allow motor to stop exactly on border and not closest int to distanceToGo/step(size)
                else:
                    self.y+=self.step # increment step
            elif event.key() == QtCore.Qt.Key_A and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.x-=self.step
                elif self.x<=0 or self.x-self.step<=0: # 
                    self.x=0
                else:
                    self.x-=self.step
            elif event.key() == QtCore.Qt.Key_S and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.y-=self.step
                elif self.y<=0 or self.y-self.step<=0:
                    self.y=0
                else:
                    self.y-=self.step
            elif event.key() == QtCore.Qt.Key_D and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.x+=self.step
                elif self.x>=self.l or self.x+self.step>=self.l:
                    self.x=self.l
                else:
                    self.x+=self.step
            elif event.key() == QtCore.Qt.Key_Shift and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.z-=self.step
                elif self.z<=0 or self.z-self.step<=0:
                    self.z=0
                else:
                    self.z-=self.step
            elif event.key() == QtCore.Qt.Key_Space and not event.isAutoRepeat():
                self.start_Serial_Move()
                if self.is_free_mode:
                    self.z+=self.step
                elif self.z>=self.h or self.z+self.step>=self.h:
                    self.z=self.h
                else:
                    self.z+=self.step
                    
            try:
                if self.serial.isOpen():
                    if not self.is_free_mode: 
                        self.serial.write(f'{self.x};{self.y};{self.z}\n'.encode('utf_8')) # write positions to serial
                    else: # antiquated free mode
                        self.serial.write(f'{self.is_key_W_pressed, self.is_key_A_pressed, self.is_key_D_pressed, self.is_key_SHIFT_pressed, self.is_key_SPACE_pressed, self.step}')
            except Exception as e: print(str(e).ljust(200), end='\r')
            self.x_display.display(self.x)
            self.y_display.display(self.y)
            self.z_display.display(self.z)
            self.z_axis.clear()
            self.main_plot.clear()
            self.z_axis.plot([0], [self.z], pen=None, symbol='o')
            self.main_plot.plot([self.x], [self.y], pen=None, symbol='o')  # setting pen=None disables line drawing
        elif event.key() == QtCore.Qt.Key_H and not event.isAutoRepeat():
            self.serial.write("H\n".encode('utf_8')) # homing program
        else:
            if not event.isAutoRepeat():
                if event.key() == QtCore.Qt.Key_W:
                    self.sendSerialPositions()
                    self.is_key_W_pressed = True # assign boolean forward                   
                elif event.key() == QtCore.Qt.Key_A:
                    self.sendSerialPositions()
                    self.is_key_A_pressed = True # assign boolean left                
                elif event.key() == QtCore.Qt.Key_S:
                    self.sendSerialPositions()
                    self.is_key_S_pressed = True # assign boolean backward                
                elif event.key() == QtCore.Qt.Key_D:
                    self.sendSerialPositions()
                    self.is_key_D_pressed = True # assign boolean right                
                elif event.key() == QtCore.Qt.Key_Space:
                    self.sendSerialPositions()
                    self.is_key_SPACE_pressed = True # assign boolean up                
                elif event.key() == QtCore.Qt.Key_Shift:
                    self.sendSerialPositions()
                    self.is_key_SHIFT_pressed = True # assign boolean down
                elif event.key() == QtCore.Qt.Key_R:
                    try:
                        if self.serial.isOpen():
                            self.serial.write('s\n'.encode('utf_8')) # stop motors
                    except Exception as e: print(str(e).ljust(200), end='\r')
                super(MainWindow, self).keyPressEvent(event)
                if any([self.is_key_W_pressed, self.is_key_A_pressed, self.is_key_S_pressed, self.is_key_D_pressed,
                           self.is_key_SHIFT_pressed, self.is_key_SPACE_pressed]):
                    self.start_move() # move if any keys are pressed
                    
    def keyReleaseEvent(self, event): # detect key release and stop movement
        if self.is_step_mode:
            pass
        else:
            if not event.isAutoRepeat():
                if event.key() == QtCore.Qt.Key_W:
                    self.sendSerialPositions()
                    self.is_key_W_pressed = False
                elif event.key() == QtCore.Qt.Key_A:
                    self.sendSerialPositions()
                    self.is_key_A_pressed = False
                elif event.key() == QtCore.Qt.Key_S:
                    self.sendSerialPositions()
                    self.is_key_S_pressed = False
                elif event.key() == QtCore.Qt.Key_D:
                    self.sendSerialPositions()
                    self.is_key_D_pressed = False
                elif event.key() == QtCore.Qt.Key_Space:
                    self.sendSerialPositions()
                    self.is_key_SPACE_pressed = False
                elif event.key() == QtCore.Qt.Key_Shift:
                    self.sendSerialPositions()
                    self.is_key_SHIFT_pressed = False
                super(MainWindow, self).keyReleaseEvent(event)
                if not any([self.is_key_W_pressed, self.is_key_A_pressed, self.is_key_S_pressed, self.is_key_D_pressed,
                           self.is_key_SHIFT_pressed, self.is_key_SPACE_pressed]):
                    self.stop_move() # stops movement if no keys are pressed, saves resources
                    try: # send last position
                        if self.serial.isOpen():
                            self.serial.write(f'P{self.x};{self.y};{self.z}\n'.encode('utf_8'))
                    except Exception as e: print(str(e).ljust(200), end='\r')                  
 
    def WASDMove(self): # regular movement
        if self.is_step_mode: # dont move if step mode is active
            pass
        else:
            self.step_x = int(self.step*self.speed.value()/100) # set step size relative to window size and speed
            self.step_y = int(self.step*self.speed.value()/100)
            self.step_z = int(self.step*self.speed.value()/100)
            if self.is_key_W_pressed: # W
                if self.is_free_mode:
                    self.y+=self.step_y # move steps
                elif self.y>=self.w or self.y+self.step_y>=self.w: # check for edge
                    self.y=self.w # edge case for if l, w, or h are not divisible by step size
                else:
                    self.y+=self.step_y # move steps

            if self.is_key_S_pressed: # S
                if self.is_free_mode:
                    self.y-=self.step_y
                elif self.y<=0 or self.y-self.step_y<=0:
                    self.y=0
                else:
                    self.y-=self.step_y

            if self.is_key_D_pressed: # D
                if self.is_free_mode:
                    self.x+=self.step_x
                elif self.x>=self.l or self.x+self.step_x>=self.l:
                    self.x=self.l
                else:
                    self.x+=self.step_x
                
            if self.is_key_A_pressed: # A
                if self.is_free_mode:
                    self.x-=self.step_x
                elif self.x<=0 or self.x-self.step_x<=0:
                    self.x=0
                else:
                    self.x-=self.step_x
                
            if self.is_key_SPACE_pressed: # SPACE
                if self.is_free_mode:
                    self.z+=self.step_z
                elif self.z>=self.h or self.z+self.step_z>=self.h:
                    self.z=self.h
                else:
                    self.z+=self.step_z
                
            if self.is_key_SHIFT_pressed: # SHIFT
                if self.is_free_mode:
                    self.z-=self.step_z
                elif self.z<=0  or self.z-self.step_z<=0:
                    self.z=0
                else:
                    self.z-=self.step_z    
            
            self.x_display.display(self.x) # clear plot
            self.y_display.display(self.y)
            self.z_display.display(self.z)
            self.z_axis.clear()
            self.z_axis.plot([0], [self.z], pen=None, symbol='o')
            self.main_plot.clear()
            try: # plot new positions
                self.main_plot.plot([self.x_actual], [self.y_actual], pen=None, symbol='x')
                self.z_axis.plot([0], [self.z_actual], pen=None, symbol='x')
            except:
                pass
            self.main_plot.plot([self.x], [self.y], pen=None, symbol='o')

    def closeEvent(self, event):
        try:
            self.serial.close()
            print(f"Closed port: {self.com_port}".ljust(200))  # Confirm port is closed
        except:
            pass
        print('\nExited')

if not QtWidgets.QApplication.instance():
    app = QtWidgets.QApplication(sys.argv)
else:
    app = QtWidgets.QApplication.instance()

if __name__ == '__main__':
    window = MainWindow()
    app.setStyle('Windows')
    window.show()
    print('Running\n')
    app.exec()

Running

Closed port:                                                                                                                                                                                                      

Exited
